In [ ]:
# Standard libraries
import sys
import re
import time
import math

# External libraries
import lightkurve as lk
import pandas as pd
import numpy as np
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.mast import Catalogs

In [ ]:
# Initial parameters
target_data_file = "../Data/RawTargetData/Angular Diameters December 2019.csv"

In [ ]:
target_list = pd.read_csv(target_data_file)
target_list

In [ ]:
"""Replace GJ IDs with HD IDs if possible"""
custom_simbad = Simbad()
custom_simbad.add_votable_fields("ids")

for idx in target_list.index.values:
    if "HD" not in target_list.loc[idx, "Star"]:
        query = custom_simbad.query_object(target_list.loc[idx, "Star"])
        ids = query["IDS"][0].decode("utf-8").split("|")
        hd = [item for item in ids if "HD" in item]
        if len(hd) > 0:
            hd = re.sub("\s\s+", " ", hd[0])
            target_list.loc[idx, "Star"] = hd

In [ ]:
def sort_ids(data_table: pd.DataFrame) -> pd.DataFrame:
    """Sort IDs by numerical order with GJ IDs first then HD IDs last.

    Parameters
    ----------
    data_table : pd.DataFrame
        the table to be sorted. Requires a Star column.
    
    Returns
    -------
    data_table : pd.DataFrame
        sorted data table
    """

    def sorting_key(entry):
        """Sorting key"""
        return entry[1]
    
    star_names = list(enumerate(data_table["Star"]))
    gj_id = []
    hd_id = []
    for star in star_names:
        if "GJ" in star[1]:
            gj_id.append(star)
        elif "HD" in star[1]:
            hd_id.append(star)

    # Sort GJ ids by number
    gj_indices = sorted([(idx, int(re.search("\d+", x[1]).group(0))) for idx, x in enumerate(gj_id)], key=sorting_key)
    gj_id = [gj_id[idx] for idx, _ in gj_indices]
    hd_indices = sorted([(idx, int(re.search("\d+", x[1]).group(0))) for idx, x in enumerate(hd_id)], key=sorting_key)
    hd_id = [hd_id[idx] for idx, _ in hd_indices]

    sorted_names = gj_id + hd_id
    sorted_indices = [idx for idx, _ in sorted_names]

    # Finally sorted
    data_table = data_table.loc[sorted_indices].reset_index(drop=True)
    return data_table


In [ ]:
# Sort the IDs
target_list = sort_ids(target_list)
print(*target_list["Star"], sep="\n")

In [ ]:
"""Filter targets with TESS data"""
# Query using lightkurve to get TIC and sectors
tic = []
sectors = []
to_add = []

start = time.time()
for idx in target_list.index.values:
    print(f"Star {idx} out of {len(target_list)}")
    star = target_list.loc[idx, "Star"]
    search = lk.search_lightcurvefile(star, mission="TESS")
    if len(search) == 0:
        continue
    else:
        to_add.append(idx)
    
    # Go through search results
    current_sectors = []
    current_target_name = ""
    # Make sure the target is the same for all sectors
    for (sector, target_name) in search.table[["observation", "target_name"]]:
        if current_target_name == "":
            current_target_name = target_name
        if target_name != current_target_name:
            continue
        current_sectors.append(sector.replace("TESS Sector ", ""))
    
    # Add new sectors and TIC
    sectors.append(", ".join(current_sectors))
    tic.append(int(current_target_name))

target_list = target_list.loc[to_add]
target_list.insert(1, "TIC", tic)
target_list.insert(2, "Sectors", sectors)
target_list.reset_index(drop=True, inplace=True)

elapsed = time.time() - start
print(f"This took {elapsed//60} minutes and {elapsed%60} seconds.")

In [ ]:
"""Clear current values and replace with SIMBAD values"""
target_list["RA"] = len(target_list) * [""]
target_list["DEC"] = len(target_list) * [""]
target_list["SpT"] = len(target_list) * [""]
target_list["V"] = len(target_list) * [np.nan]
target_list["H"] = len(target_list) * [np.nan]
target_list["K"] = len(target_list) * [np.nan]
target_list.insert(1, "Main ID", len(target_list) * ["N/A"])

In [ ]:
custom_simbad = Simbad()
custom_simbad.add_votable_fields(
    "sptype",
    "flux(V)", "flux_bibcode(V)",
    "flux(H)", "flux_bibcode(H)",
    "flux(K)", "flux_bibcode(K)"
)

In [ ]:
query_delay = 3.0
minimum_time = query_delay*len(target_list["Star"])
print(f"Querying will at least {minimum_time//60} minutes and {minimum_time%60} seconds.")

# SIMBAD querying
for idx in target_list.index.values:
    # Query until success
    query = None
    while query is None:
        try:
            query = custom_simbad.query_object(target_list.loc[idx, "Star"])
            print(f"Successfully queried #{idx+1}: {target_list.loc[idx, 'Star']}")
        except:
            print(f"Error: Retrying connection...", end="\r")
            time.sleep(5.0)
    
    # Process query
    main_id = re.sub("\s\s+", " ", query["MAIN_ID"][0].decode("utf-8"))
    ra = query["RA"][0]
    dec = query["DEC"][0]
    spt = query["SP_TYPE"][0].decode("utf-8")
    # Fluxes
    v_mag = query["FLUX_V"][0]
    h_mag = query["FLUX_H"][0]
    k_mag = query["FLUX_K"][0]

    # Set new values
    target_list.loc[idx, "Main ID"] = re.sub("^\*\s*", "", main_id)
    target_list.loc[idx, "RA"] = ra
    target_list.loc[idx, "DEC"] = dec
    target_list.loc[idx, "SpT"] = spt
    target_list.loc[idx, "V"] = v_mag
    target_list.loc[idx, "H"] = h_mag
    target_list.loc[idx, "K"] = k_mag

    time.sleep(query_delay)
target_list

In [ ]:
"""Add fast cadences"""
target_list.insert(len(target_list.columns), "Fast", len(target_list)*[False])
target_list_append = target_list[0:0]
for idx in target_list.index.values:
    search = lk.search_lightcurvefile(target_list.loc[idx, "Star"], mission="TESS")
    
    fast_sectors = []

    sectors = []
    current_target = None
    for sector, target, filename in search.table[["observation", "target_name", "productFilename"]]:            
        if current_target is None:
            current_target = target
        elif current_target != target:
            continue

        if "fast" in filename:
            fast_sectors.append(sector.replace("TESS Sector ", ""))
        else:
            sectors.append(sector.replace("TESS Sector ", ""))
    
    target_list.loc[idx, "Sectors"] = ", ".join(sectors)

    if len(fast_sectors) > 0:
        current_fast_index = len(target_list_append)
        target_list_append.loc[current_fast_index] = target_list.loc[idx]
        target_list_append.loc[current_fast_index, "Sectors"] = ", ".join(fast_sectors)
        target_list_append.loc[current_fast_index, "Fast"] = True
    print(f"Star {idx} out of {len(target_list)}")
target_list = pd.concat([target_list, target_list_append], ignore_index=True)
target_list = sort_ids(target_list)
target_list.reset_index(drop=True, inplace=True)
target_list

In [ ]:
"""Add ATL Pmix and numax"""

target_list.insert(4, "Pmix", len(target_list) * [np.nan])
target_list.insert(5, "ATL numax (muHz)", len(target_list) * [np.nan])

# Search ATL catalogue
v = Vizier(columns=["TIC", "Pmix", "numax"], catalog="J/ApJS/241/12")

for idx in target_list.index.values:
    star = target_list.loc[idx, "Star"]
    search = v.query_object(star)
    if len(search) == 0:
        pmix = np.nan
        numax = np.nan
    else:
        search = search[0]
        pmix = float(search["Pmix"][0])
        numax = float(search["numax"][0])
    target_list.loc[idx, "Pmix"] = pmix
    target_list.loc[idx, "ATL numax (muHz)"] = numax
    print(f"Star {idx} out of {len(target_list)}")

target_list

In [ ]:
"""TIC estimated numax"""
def estimate_numax(tic_number, mass):
    result = Catalogs.query_criteria(catalog="Tic", ID=tic_number)
    t_eff = result["Teff"][0]
    radius = result["rad"][0]
    # Stefan-Boltzmann law
    luminosity = radius**2 * (t_eff/5777.0)**4
    # Estimate numax from scaling relation
    numax = 3100.0 * mass * (t_eff/5777.0)**3.5 / luminosity
    return numax

# Add new columns
target_list.insert(6, "TIC 0.8M numax (muHz)", len(target_list) * [np.nan])
target_list.insert(7, "TIC 1.2M numax (muHz)", len(target_list) * [np.nan])
target_list.insert(8, "TIC 2.4M numax (muHz)", len(target_list) * [np.nan])

for idx in target_list.index.values:
    spt = target_list.loc[idx, "SpT"]
    if "F" in spt or "G" in spt or "K" in spt:
        tic_number = target_list.loc[idx, "TIC"]
        target_list.loc[idx, "TIC 0.8M numax (muHz)"] = estimate_numax(tic_number, 0.8)
        target_list.loc[idx, "TIC 1.2M numax (muHz)"] = estimate_numax(tic_number, 1.2)
        target_list.loc[idx, "TIC 2.4M numax (muHz)"] = estimate_numax(tic_number, 2.4)
    print(f"Star {idx} out of {len(target_list)}")
target_list

In [ ]:
target_list.to_csv("../Data/PlotData/dht_plot_data.csv", index=False)